In [3]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
train_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageNet(root='../data/imagenet-mini', split='val', transform=val_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
model_path = '../models/model_MLP_Modified_scripted.pt'
model = torch.jit.load(model_path, map_location=device).to(device)

ValueError: The provided filename ../models/model_MLP_Modified_scripted.pt does not exist

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),eps=1e-08, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5//2, verbose=True)

NameError: name 'model' is not defined

In [ ]:
num_epochs = 50

In [ ]:
log_df = pd.DataFrame(columns=['epoch', 'accuracy', 'val_loss'])

In [ ]:
for epoch in tqdm(range(num_epochs), desc="Total Training Progress"):
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()  

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    scheduler.step(val_loss)  # Adjust learning rate based on the validation loss

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(val_loader):.4f}, Accuracy: {accuracy:.2f}%, Validation Loss: {val_loss/len(val_loader):.4f}')

    log_df = log_df.append({'epoch': epoch+1, 'accuracy': accuracy, 'val_loss': val_loss/len(val_loader)}, ignore_index=True)

log_df.to_csv('training_log.csv', index=False)

scripted_trained_model = torch.jit.script(model)
scripted_trained_model.save("trained_models/trained_model_scripted_modified_MLP.pt")